# Key generation 

In [1]:
import cirq
from random import choices

In [2]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H} 

num_bits = 2000
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

In [3]:
alice_key = choices([0,1], k =num_bits)

#print('Alice\'s initial key: ', alice_key)

In [4]:
alice_bases = choices(['Z', 'X'], k =num_bits)

#print('\nAlice\'s randomly chosen bases: ', alice_bases)

In [5]:
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

In [6]:
bob_bases = choices(['Z', 'X'], k=num_bits)


bob_circuit = cirq.Circuit()

for bit in range (num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

In [7]:
bob_circuit.append(cirq.measure(qubits, key = 'bob key') )

#print(bob_circuit)

In [8]:
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

#print('\nBob\'s initial key: ', bob_key)

In [9]:
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

#print('\nAlice\'s key: ', final_alice_key)
#print('Bob\'s key: ', final_bob_key)

In [10]:
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



We can use our keys!
Alice Key:  [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0

# Encryption

In [11]:
def concat_ints(key_list):
    # Convert each integer in the list to a string and concatenate
    concatenated_string = ''.join(str(num) for num in key_list)
    return concatenated_string


In [12]:
def text_to_binary(text):
    # Converts text to 8-bit binary for each character
    return ''.join(format(ord(char), '08b') for char in text)

In [13]:
def xor_encrypt(message, key):
    # Check if the key is long enough to encrypt the message (i.e. not shorter than the message)
    if len(key) < len(message):
        print ('ERROR!! Key is too short.\n SOLUTION: Use a shorter message. \n OR Use a higher num_bits in the key generation for the decryptor to work.')
    else:
        encrypted_binary = ''.join('1' if message[i] != key[i] else '0' for i in range(len(message)))
        return encrypted_binary


In [14]:
def binary_to_text(binary):
    # Converts encrypted binary message to an encrypted text mesage
    return ''.join(chr(int(binary[i:i+8], 2)) for i in range(0, len(binary), 8))


In [15]:
def encryptor ():
    
    initial_message = input("Please enter the message you want to encrypt and send:")
    message_binary = text_to_binary(initial_message)
    key_list = final_alice_key
    final_key = concat_ints(key_list)
    encrypted = xor_encrypt(message_binary, final_key)
    final_encrypted = binary_to_text(encrypted)
    
    return final_encrypted

In [17]:
encryptor()

Please enter the message you want to encrypt and send: I love quantum mechanics.


'¹D/±þ\x12CNQL«B-\x18\x07Y°\x0b\x14Ö~\tåê\x8b'

# Decryption

In [30]:
def decryptor (received_message):
    received_message_binary = text_to_binary(received_message)
    key_list = final_bob_key
    final_key = concat_ints(key_list)
    decrypted = xor_encrypt(received_message_binary, final_key)
    final_decrypted = binary_to_text(decrypted)
    return final_decrypted

In [31]:
decryptor('¹D/±þ\x12CNQL«B-\x18\x07Y°\x0b\x14Ö~\tåê\x8b')

'I love quantum mechanics.'